# Experiment: aerodynamic instability

Here I'll work on the experiment of chapter 6 of the ODE book I'm following.

v_c = sqrt(52/6). If v > v_c we get unstable solutions, if v < v_c the solutions are stable.

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from typing import List, Tuple
import warnings
from dataclasses import dataclass
import math

### I <3 Claude

In [6]:
@dataclass
class EigenBranch:
    """Class to store and track eigenvalue branches."""
    value: complex
    branch_type: str  # 'outer' or 'inner'
    sign: int        # +1 or -1

def compute_sqrt_with_tracking(z: complex, prev_value: complex = None) -> complex:
    """
    Compute square root while maintaining continuity with previous value.
    
    Args:
        z (complex): Value to compute square root of
        prev_value (complex): Previous square root value for continuity
        
    Returns:
        complex: Square root value maintaining continuity
    """
    # Compute both possible square root values
    sqrt1 = np.sqrt(z)
    sqrt2 = -sqrt1
    
    if prev_value is None:
        return sqrt1
        
    # Choose the value that maintains continuity
    if abs(sqrt1 - prev_value) < abs(sqrt2 - prev_value):
        return sqrt1
    return sqrt2

def compute_eigenvalues(v: float, prev_values: List[EigenBranch] = None) -> List[EigenBranch]:
    """
    Compute the four eigenvalues for a given value of v with branch tracking.
    
    Args:
        v (float): The parameter value
        prev_values (List[EigenBranch]): Previous eigenvalues for continuity
        
    Returns:
        List[EigenBranch]: The four eigenvalues with branch information
    """
    v = complex(v)
    discriminant = 52 - 6 * v**2
    
    # For the outer square root (sqrt(-8 Â± sqrt(discriminant)))
    if prev_values is None:
        # Initial computation without previous values
        sqrt_discr = np.sqrt(discriminant + 0j)
        
        # Compute both outer branches
        outer1 = np.sqrt(-8 + sqrt_discr)
        outer2 = np.sqrt(-8 - sqrt_discr)
        
        return [
            EigenBranch(outer1, 'outer', 1),
            EigenBranch(-outer1, 'outer', -1),
            EigenBranch(outer2, 'outer', 1),
            EigenBranch(-outer2, 'outer', -1)
        ]
    else:
        # Compute with continuity
        sqrt_discr = compute_sqrt_with_tracking(
            discriminant + 0j,
            prev_values[0].value**2 + 8  # Use previous value to track the branch
        )
        
        # Track both outer branches separately
        outer1 = compute_sqrt_with_tracking(-8 + sqrt_discr, prev_values[0].value)
        outer2 = compute_sqrt_with_tracking(-8 - sqrt_discr, prev_values[2].value)
        
        return [
            EigenBranch(outer1, 'outer', 1),
            EigenBranch(-outer1, 'outer', -1),
            EigenBranch(outer2, 'outer', 1),
            EigenBranch(-outer2, 'outer', -1)
        ]

def generate_eigenvalue_data(v_min: float, v_max: float, 
                           num_points: int = 2000) -> Tuple[np.ndarray, List[np.ndarray]]:
    """
    Generate eigenvalue data over a range of v values with branch tracking.
    
    Args:
        v_min (float): Minimum value of v
        v_max (float): Maximum value of v
        num_points (int): Number of points to evaluate
        
    Returns:
        Tuple[np.ndarray, List[np.ndarray]]: v values and list of eigenvalue arrays
    """
    v_values = np.linspace(v_min, v_max, num_points)
    eigenvalues = [np.zeros(num_points, dtype=complex) for _ in range(4)]
    
    # Initial computation
    prev_branches = None
    
    # Compute eigenvalues for each v with branch tracking
    for i, v in enumerate(v_values):
        branches = compute_eigenvalues(v, prev_branches)
        for j in range(4):
            eigenvalues[j][i] = branches[j].value
        prev_branches = branches
            
    return v_values, eigenvalues

def create_3d_plot(v_values: np.ndarray, eigenvalues: List[np.ndarray], 
                  v_crit: float) -> plt.Figure:
    """
    Create a 3D visualization of the eigenvalues.
    
    Args:
        v_values (np.ndarray): Array of v values
        eigenvalues (List[np.ndarray]): List of eigenvalue arrays
        v_crit (float): Critical value of v
        
    Returns:
        plt.Figure: The matplotlib figure object
    """
    fig = plt.figure(figsize=(12, 8), dpi=100)
    ax = fig.add_subplot(111, projection='3d')
    
    colors = ['navy', 'cyan', 'red', 'magenta']
    labels = ['$\\lambda_{}$'.format(i+1) for i in range(4)]
    
    # Plot each eigenvalue curve
    for i, (eig, color, label) in enumerate(zip(eigenvalues, colors, labels)):
        ax.plot(v_values, eig.imag, eig.real, 
                color=color, 
                linewidth=2, 
                label=label)
    
    # Add a vertical line at v_crit
    ax.plot([v_crit, v_crit], 
            [min(min(eig.imag) for eig in eigenvalues), 
             max(max(eig.imag) for eig in eigenvalues)],
            [0, 0], 
            'k--', 
            alpha=0.5,
            label='$v_{crit}$')
    
    # Customize the appearance
    ax.set_xlabel('$v$', fontsize=14, labelpad=10)
    ax.set_ylabel('$\\Im(\\lambda)$', fontsize=14, labelpad=10)
    ax.set_zlabel('$\\Re(\\lambda)$', fontsize=14, labelpad=10)
    ax.set_title('Eigenvalue Spectrum vs Parameter $v$', fontsize=16, pad=20)
    
    # Set axis limits with margin
    margin = 0.1
    ax.set_xlim([0, v_crit * 2])
    y_min = min(min(eig.imag) for eig in eigenvalues)
    y_max = max(max(eig.imag) for eig in eigenvalues)
    z_min = min(min(eig.real) for eig in eigenvalues)
    z_max = max(max(eig.real) for eig in eigenvalues)
    
    ax.set_ylim([y_min * (1 + margin), y_max * (1 + margin)])
    ax.set_zlim([z_min * (1 + margin), z_max * (1 + margin)])
    
    # Enhance grid and viewing angle
    ax.grid(True, linestyle='--', alpha=0.3)
    ax.view_init(elev=20, azim=45)
    
    # Add legend with background
    ax.legend(loc='upper right', fontsize=12, 
             bbox_to_anchor=(0.98, 0.98),
             bbox_transform=ax.transAxes,
             fancybox=True, 
             shadow=True)
    
    return fig

def main():
    """Main function to generate and display the plot."""
    warnings.filterwarnings('ignore')
    
    # Define constants
    V_CRIT = np.sqrt(52 / 6)
    
    # Generate data with increased resolution around v_crit
    v_values, eigenvalues = generate_eigenvalue_data(0, 2 * V_CRIT)
    
    # Create and show the plot
    fig = create_3d_plot(v_values, eigenvalues, V_CRIT)
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()

### Methods

In [7]:
# Lists with Butcher tableau for RKF45
A = np.array([
    [1/4, 0, 0, 0, 0],
    [3/32, 9/32, 0, 0, 0],
    [1932/2197, -7200/2197, 7296/2197, 0, 0],
    [439/216, -8, 3680/513, -845/4104, 0],
    [-8/27, 2, -3544/2565, 1859/4104, -11/40]
])
B4 = np.array([25/216, 0, 1408/2565, 2197/4104, -1/5, 0])
B5 = np.array([16/135, 0, 6656/12825, 28561/56430, -9/50, 2/55])
C = np.array([0, 1/4, 3/8, 12/13, 1, 1/2])

# Bounds so algorithm always stops running
MAX_ERROR = 1e-6
EPSILON = 1e-15

# Dynamically compute the stepsize (h)
def RKF45_step(F, t, x, h, max_iterations=10):
	dim = len(x)
	l = len(C)
	k = np.zeros((l, dim))
	k[0] = F(t, x)
	
	for _ in range(max_iterations):
		for i in range(1, l):
			k[i] = F(t + h * C[i], x + h * np.dot(A[i-1, :i], k[:i]))
		# Compute next time
		t_next = t + h # is this correct? should't t_next be based on h_next? no, this here is correct

		# Compute next x with order 4 and 5 methods
		x4 = x + h * np.dot(B4, k)
		x5 = x + h * np.dot(B5, k)

		# Compute error
		error = np.linalg.norm(x4 - x5)

		# Update h
		h *= np.power(MAX_ERROR / (2 * max(error, EPSILON)), 1/4)
		
		# Break if error is below threshold
		if error <= MAX_ERROR:
			break

	return t_next, x4, h

# The main function
def RKF45(F, t0, x0, h0, t_final):
    # Initialize stuff
    t, x, h = [t0], [x0], [h0]

    while t[-1] < t_final:
        t_next, x_next, h_next = RKF45_step(F, t[-1], x[-1], h[-1])
        t.append(t_next)
        x.append(x_next)
        h.append(h_next)
        
    return np.array(t), np.vstack(x), np.array(h)

### ODE and solution

In [8]:
def aero(t, X, v):
	x, y, dx, dy = X
	Dx = dx
	Dy = dy
	Ddx = -4 * x + (6 - v**2) * y
	Ddy = 6 * x - 12 * y
	return np.array([Dx, Dy, Ddx, Ddy])

In [9]:
V_CRIT = math.sqrt(52 / 6)

In [10]:
def aero_0(t, X):
	return aero(t, X, 0)

def aero_crit(t, X):
	return aero(t, X, V_CRIT)

def aero_supercrit(t, X):
	return aero(t, X, V_CRIT + 0.1)

In [16]:
t0 = 0
x0 = np.array([1., 2., 0., 0.])
h0 = 0.01
t_final = 10.

t_0, X_0, _ = RKF45(aero_0, t0, x0, h0, t_final)
t_crit, X_crit, _ = RKF45(aero_crit, t0, x0, h0, t_final)
t_supercrit, X_supercrit, _ = RKF45(aero_supercrit, t0, x0, h0, t_final)

In [17]:
plt.plot(t_0, X_0[:, 0])
plt.plot(t_crit, X_crit[:, 0])
plt.plot(t_supercrit, X_supercrit[:, 0])

In [18]:
plt.plot(t_0, X_0[:, 1])
plt.plot(t_crit, X_crit[:, 1])
plt.plot(t_supercrit, X_supercrit[:, 1])